In [67]:
import os
import pandas as pd
import glob
import numpy as np
_dfs_list = []
counter = 0
for csv_filename in glob.glob('archive/SMNI_CMI_TRAIN/*.csv'):
#     if counter == 100:
#         break
    _dfs_list.append(pd.read_csv(csv_filename))
    counter += 1
    if counter == 1:
        print(len(_dfs_list[0]))
df = pd.concat(_dfs_list)
del(_dfs_list)
df = df.drop(['Unnamed: 0'], axis=1)
df.head(3)

16384


,trial number,sensor position,sample num,sensor value,subject identifier,matching condition,channel,name,time
0,0,FP1,0,-8.921,a,S1 obj,0,co2a0000364,0.000000
1,0,FP1,1,-8.433,a,S1 obj,0,co2a0000364,0.003906
2,0,FP1,2,-2.574,a,S1 obj,0,co2a0000364,0.007812


In [65]:
df[df['name'] == 'co2a0000364']

,trial number,sensor position,sample num,sensor value,subject identifier,matching condition,channel,name,time
0,0,FP1,0,-8.921,a,S1 obj,0,co2a0000364,0.000000
1,0,FP1,1,-8.433,a,S1 obj,0,co2a0000364,0.003906
2,0,FP1,2,-2.574,a,S1 obj,0,co2a0000364,0.007812
3,0,FP1,3,5.239,a,S1 obj,0,co2a0000364,0.011719
4,0,FP1,4,11.587,a,S1 obj,0,co2a0000364,0.015625
...,...,...,...,...,...,...,...,...,...
16379,17,Y,251,-3.072,a,S2 match,63,co2a0000364,0.980469
16380,17,Y,252,1.811,a,S2 match,63,co2a0000364,0.984375
16381,17,Y,253,4.740,a,S2 match,63,co2a0000364,0.988281
16382,17,Y,254,4.252,a,S2 match,63,co2a0000364,0.992188


In [15]:
a_df = df[df['subject identifier']=='a']
c_df = df[df['subject identifier']=='c']

In [55]:
counter

468

In [3]:
print('dh')

dh


In [24]:
df['sensor value'].min()

-161.296

In [157]:
import random
from tqdm import tqdm

class TS_hd_classifier():
 
    def __init__(self, window_size=3, dim=2000):
        self.dim = dim
        self.label_len = 2
        self.channel_num = 64
        self.sample_per_channel = 256
        self.window_size = window_size
        
        #todo: fix max and min calc 
        self.max_sensor_val = 30
        self.min_sensor_val = -30
        
        
        self.level_hv_min = self.random_hv_generator()
        self.level_hv_max = self.random_hv_generator()
        
        # initializing middle level-hypervectors
        self.level_hvs = [None for i in range(9)]
        for i in range(len(self.level_hvs)):
            self.level_hvs[i] = [0 for i in range(self.dim)]
            ratio = (i+1)/10
            for j in range(self.dim):
                if random.uniform(0, 1) < ratio:
                    self.level_hvs[i][j] = self.level_hv_max[j]
                else:
                    self.level_hvs[i][j] = self.level_hv_min[j]
                    
        # bundled classifier hv for every label in perm-based encoding (0: non-alcoholic, 1: alcoholic)
        self.classes_encodings = np.asarray([[0 for i in range(self.dim)] for j in range(self.label_len)])
        
        # generate identification vectors for associating different channels
        self.id_channel = [None for i in range(self.channel_num)]
        for i in range(len(self.id_channel)):
            self.id_channel[i] = self.random_hv_generator()
    
    def assign_level_hv(self, e):
        result = None
        if e <= -1:
            return self.level_hv_min
        elif e >= 1:
            return self.level_hv_max
        else:
            ratio = ((e + 1)/2)*10
            if 0 <= ratio < 0.5:
                result = self.level_hv_min
            elif 0.5 <= ratio < 1.5:
                result = self.level_hvs[0]
            elif 1.5 <= ratio < 2.5:
                result = self.level_hvs[1]
            elif 2.5 <= ratio < 3.5:
                result = self.level_hvs[2]
            elif 3.5 <= ratio < 4.5:
                result = self.level_hvs[3]
            elif 4.5 <= ratio < 5.5:
                result = self.level_hvs[4]
            elif 5.5 <= ratio < 6.5:
                result = self.level_hvs[5]
            elif 6.5 <= ratio < 7.5:
                result = self.level_hvs[6]
            elif 7.5 <= ratio < 8.5:
                result = self.level_hvs[7]
            elif 8.5 <= ratio < 9.5:
                result = self.level_hvs[8]
            else:
                result = self.level_hv_max
        
        return result
    
    def encode_ts(self, csv_file):
        df = pd.read_csv(csv_file)
        sample = df['sensor value'].to_numpy()
        sample = self.normalize(sample)
        
        # 0: non-alc, 1: alc
        label = 0 if df['subject identifier'].iloc[0] == 'c' else 1
        
#         A = np.asarray([0 for i in range(self.dim)])
        
        for i in range(self.channel_num):
            channel_samples = sample[i*256: i*256 + 256]
            M = np.asarray([0 for i in range(self.dim)])
            
            for j in range(len(channel_samples) - self.window_size):
                current_window = channel_samples[j:j+self.window_size]
                encoded_window = self.encode_window(current_window)
                M = np.add(M, encoded_window)
            
            # now we should first bind M with its corresbonding channel id_hv and then bundle it with the label class hv
            M = np.multiply(M, self.id_channel[i])
            
        return M, label
        
    
    def encode_window(self, window):
        H = np.asarray([1 for i in range(self.dim)])
        for i in range(self.window_size):
            encoded_val = self.encode_value(window[i])
            encoded_val = self.perm_hv(encoded_val, self.window_size - i)
            H = np.multiply(H, encoded_val)
        return H
    
    def encode_value(self, val):
        
        return self.assign_level_hv(val)
        
        
        
    def normalize(self, sample):
        norm_sample = []
        for e in sample:
            norm_e = (e - self.min_sensor_val)/(self.max_sensor_val - self.min_sensor_val)
            norm_sample.append(norm_e)
        return np.asarray(norm_sample)
        
    
    def random_hv_generator(self):
        hv = np.asarray([1 for i in range(self.dim)])
        for i in range(self.dim):
            if random.randint(0, 1) == 0:
                hv[i] = -1
        return hv
    

    def perm_hv(self, hv, perm_num):
        return np.roll(hv, perm_num)
  
    def cosine(self, encoded_csv, classes):
        pred_idx = -1
        cosine_val = 0
        for i in range(self.label_len):
            sim_val = np.dot(encoded_csv, classes[i])
            if sim_val > cosine_val:
                pred_idx = i
                cosine_val = sim_val

        return pred_idx

    def predict(self, csv_file):
        encoded_csv, correct_label = self.encode_ts(csv_file)
        predicted_class = self.cosine(encoded_csv, self.classes_encodings)
        return predicted_class, predicted_class == correct_label
        
    def run(self):
        
        for csv_filename in glob.glob('archive/SMNI_CMI_TRAIN/*.csv'):
            csv_file_number = int(csv_filename.split('\\')[1][4:-4])
#             print(csv_file_number)
            if csv_file_number >= 200 and csv_file_number <= 270:
#                 print(csv_file_number)
                encoded_csv, label = self.encode_ts(csv_filename)
                self.classes_encodings[label] = np.add(self.classes_encodings[label], encoded_csv)
        return

In [152]:
ts = TS_hd_classifier()

ts.run()

In [154]:
ts.classes_encodings[0]

array([ 3497, -3503, -1183, ..., -3275, -8855,  2163])

In [155]:
total = 0
counter = 0
for csv_filename in glob.glob('archive/SMNI_CMI_TEST/*.csv'):
    num = int(csv_filename.split('\\')[1][4:-4])
#     print(num)
    if 200 <= num <= 270:
#         print(num)
        total += 1
        pred, correct_or_not = ts.predict(csv_filename)
#         print(pred)
        if correct_or_not:
            counter += 1
accuracy = counter/total
print(accuracy)


0.676056338028169


### Part 1, accuracy for training time series data with dim=2000 and training and testing on about 70 csv files (each with 64 channels and 255 samples per channel) is about 68 percent.

## Part 2: Using different sampling windows

In [160]:
def test_model(model):
    total = 0
    counter = 0
    for csv_filename in glob.glob('archive/SMNI_CMI_TEST/*.csv'):
        num = int(csv_filename.split('\\')[1][4:-4])
        if 210 <= num <= 265:
            total += 1
            pred, correct_or_not = model.predict(csv_filename)
            if correct_or_not:
                counter += 1
    accuracy = counter/total
    return accuracy

In [161]:
acc_windows = []
# sampling_windows = [3, 4, 5]
ts_hd_classifiers = [TS_hd_classifier(i+3) for i in range(3)]
for i in range(len(ts_hd_classifiers)):
    model = ts_hd_classifiers[i]
    print('training model...')
    model.run()
    print('done! calculating accuracy on test data')
    acc = test_model(model)
    print('accuracy for window_size ' + str(i + 3) + ' is:')
    print(acc)
    acc_windows.append(acc)

training model...
done! calculating accuracy on test data
accuracy for window_size 3 is:
0.625
training model...
done! calculating accuracy on test data
accuracy for window_size 4 is:
0.6785714285714286
training model...
done! calculating accuracy on test data
accuracy for window_size 5 is:
0.6607142857142857


|Window size   | accuracy |
|----------|---------------|
| 3 | %62.50 |
| 4 | %67.76 |
| 5 | %66.07 |

#### As we can see, increasing window size generally provides better accuracy.

## Part 3: Using different dimentionalities


In [ ]:
acc_dims = []
ts_hd_classifiers_1 = [TS_hd_classifier(dim=(i+1)*1000) for i in range(3)]
for i in range(len(ts_hd_classifiers_1)):
    model = ts_hd_classifiers_1[i]
    print('training model...')
    model.run()
    print('done! calculating accuracy on test data')
    acc = test_model(model)
    print('accuracy for dim ' + str((i+1)*1000) + ' is:')
    print(acc)
    acc_dims.append(acc)

training model...
